## Importación Librerias

In [1]:
import os
import cv2
import imageio
import numpy as np
import pandas as pd
from typing import List
from utils import non_max_suppression, get_hsv_color_ranges

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils import *
from bow import BoW
from dataset import Dataset
from image_classifier import ImageClassifier
import time
from tqdm import tqdm
import sys
import pickle

## Carga del dataset

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahemateja19bec1025/traffic-sign-dataset-classification")

print("Path to dataset files:", path)


Path to dataset files: C:\Users\meryg\.cache\kagglehub\datasets\ahemateja19bec1025\traffic-sign-dataset-classification\versions\2


In [15]:
import shutil
num_to_label = pd.read_csv("../data/labels.csv", index_col="ClassId").to_dict()["Name"]
num_to_label = {str(k): v for k,v in num_to_label.items()}
num_to_label = {str(k): v.replace('/', '') for k, v in num_to_label.items()}
print(num_to_label)

{'0': 'Speed limit (5kmh)', '1': 'Speed limit (15kmh)', '2': 'Speed limit (30kmh)', '3': 'Speed limit (40kmh)', '4': 'Speed limit (50kmh)', '5': 'Speed limit (60kmh)', '6': 'Speed limit (70kmh)', '7': 'speed limit (80kmh)', '8': 'Dont Go straight or left', '9': 'Dont Go straight or Right', '10': 'Dont Go straight', '11': 'Dont Go Left', '12': 'Dont Go Left or Right', '13': 'Dont Go Right', '14': 'Dont overtake from Left', '15': 'No Uturn', '16': 'No Car', '17': 'No horn', '18': 'Speed limit (40kmh)', '19': 'Speed limit (50kmh)', '20': 'Go straight or right', '21': 'Go straight', '22': 'Go Left', '23': 'Go Left or right', '24': 'Go Right', '25': 'keep Left', '26': 'keep Right', '27': 'Roundabout mandatory', '28': 'watch out for cars', '29': 'Horn', '30': 'Bicycles crossing', '31': 'Uturn', '32': 'Road Divider', '33': 'Traffic signals', '34': 'Danger Ahead', '35': 'Zebra Crossing', '36': 'Bicycles crossing', '37': 'Children crossing', '38': 'Dangerous curve to the left', '39': 'Dangerous

In [16]:
train_data_dir = "../data/traffic_Data/DATA/"
for dir in os.listdir(train_data_dir):
    if dir in num_to_label.keys():
        dir_path = os.path.join(train_data_dir, dir)
        new_dir_path = os.path.join(train_data_dir, num_to_label[dir])
        print(f"Renaming {dir_path} to {new_dir_path}")
        if os.path.exists(dir_path):
            if os.path.exists(new_dir_path):
                shutil.move(dir_path, new_dir_path)
            else:
                os.renames(dir_path, new_dir_path)
        else:
            print(f"Path {dir_path} does not exist")

test_data_dir = "../data/traffic_Data/TEST/"

for image in os.listdir(test_data_dir):
    num = image[:3]
    try:
        num = str(int(num))
        label = num_to_label[num]
        dir_path = os.path.join(test_data_dir, label)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

        image_path = os.path.join(test_data_dir, image)
        shutil.move(image_path, dir_path)
    except:
        pass

Renaming ../data/traffic_Data/DATA/0 to ../data/traffic_Data/DATA/Speed limit (5kmh)
Renaming ../data/traffic_Data/DATA/1 to ../data/traffic_Data/DATA/Speed limit (15kmh)
Renaming ../data/traffic_Data/DATA/10 to ../data/traffic_Data/DATA/Dont Go straight
Renaming ../data/traffic_Data/DATA/11 to ../data/traffic_Data/DATA/Dont Go Left
Renaming ../data/traffic_Data/DATA/12 to ../data/traffic_Data/DATA/Dont Go Left or Right
Renaming ../data/traffic_Data/DATA/13 to ../data/traffic_Data/DATA/Dont Go Right
Renaming ../data/traffic_Data/DATA/14 to ../data/traffic_Data/DATA/Dont overtake from Left
Renaming ../data/traffic_Data/DATA/15 to ../data/traffic_Data/DATA/No Uturn
Renaming ../data/traffic_Data/DATA/16 to ../data/traffic_Data/DATA/No Car
Renaming ../data/traffic_Data/DATA/17 to ../data/traffic_Data/DATA/No horn
Renaming ../data/traffic_Data/DATA/18 to ../data/traffic_Data/DATA/Speed limit (40kmh)
Renaming ../data/traffic_Data/DATA/19 to ../data/traffic_Data/DATA/Speed limit (50kmh)
Renam

In [22]:
# for dir in os.listdir(train_data_dir):
#     print(dir, ":",len(os.listdir(os.path.join(train_data_dir, dir))))

In [4]:
# Cargar conjuntos de datos
training_set = Dataset.load("../data/traffic_Data/DATA", "*png")
validation_set = Dataset.load("../data/traffic_Data/TEST", "*png")

print(training_set[0])
print(validation_set[0])

../data/traffic_Data/DATA\Bicycles crossing\030_0001.png
../data/traffic_Data/TEST\Bicycles crossing\030_0001_j.png


In [5]:
# Crear el extractor de características SIFT
feature_extractor = cv2.SIFT_create()

# Extraer descriptores
print("\nComputing SIFT descriptors...")
time.sleep(0.1)  # Previene problemas de concurrencia entre tqdm y print

descriptors = []  # Lista para almacenar descriptores
for path in tqdm(training_set, unit="image", file=sys.stdout):
    # Cargar la imagen en escala de grises
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    try:
        # Detectar y describir características SIFT
        keypoints, descriptor = feature_extractor.detectAndCompute(image, None)
        if descriptor is not None:
            descriptors.append(descriptor)
        else:
            print(f"Advertencia: No se encontraron descriptores en {path}")
    except Exception as e:
        print(f"Error procesando {path}: {e}")
print(f"Número de descriptores extraídos: {len(descriptors)}")


# Define constants
vocabulary_size = 200
iterations = 20
termination_criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, iterations, 1e-6)

# Initialize BOWKMeansTrainer
words = cv2.BOWKMeansTrainer(vocabulary_size, termination_criteria)

# Add descriptors to the trainer
for desc in tqdm(descriptors, desc="Adding descriptors"):
    words.add(desc)

time.sleep(0.1)  # Prevent tqdm printing issues
print("\nClustering descriptors into", vocabulary_size, "words using K-means...")

# Perform k-means clustering to build the vocabulary
vocabulary = words.cluster()
print(f"Vocab: {vocabulary}")
filename=  "vocabulary.pickle"
# TODO: Open the file from above in the write and binay mode
with open(filename,"wb") as f:
    pickle.dump(["SIFT", vocabulary], f, pickle.HIGHEST_PROTOCOL)
    
try:
    with open("vocabulary.pickle", "rb") as f:
        data = pickle.load(f)
        print(f"Data from vocab.pickle: {data}")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")





Computing SIFT descriptors...
 47%|████▋     | 1158/2474 [00:10<00:10, 127.80image/s]Advertencia: No se encontraron descriptores en ../data/traffic_Data/DATA\No stopping\054_0084.png
Advertencia: No se encontraron descriptores en ../data/traffic_Data/DATA\No stopping\054_0085.png
 54%|█████▎    | 1328/2474 [00:11<00:08, 130.45image/s]Advertencia: No se encontraron descriptores en ../data/traffic_Data/DATA\No stopping\054_1_0084.png
Advertencia: No se encontraron descriptores en ../data/traffic_Data/DATA\No stopping\054_1_0085.png
100%|██████████| 2474/2474 [00:23<00:00, 106.00image/s]
Número de descriptores extraídos: 2460


Adding descriptors: 100%|██████████| 2460/2460 [00:00<00:00, 490725.19it/s]


Clustering descriptors into 200 words using K-means...


Vocab: [[26.589176  19.646986  16.05658   ...  7.7675276 20.476015  82.80566  ]
 [20.429173  16.325386  18.678822  ...  8.953716  10.32258   13.795231 ]
 [12.375405  69.00971   41.239483  ...  8.831716   5.223301   7.893204 ]
 ...
 [32.826954  18.085758  11.012252  ... 16.411945  17.134764  20.413477 ]
 [39.266953  12.705211   8.162741  ... 13.226267   4.980728   5.700214 ]
 [12.268742   9.340878  11.239038  ...  3.8062236  5.8656297  7.2560115]]
Data from vocab.pickle: ['SIFT', array([[26.589176 , 19.646986 , 16.05658  , ...,  7.7675276, 20.476015 ,
        82.80566  ],
       [20.429173 , 16.325386 , 18.678822 , ...,  8.953716 , 10.32258  ,
        13.795231 ],
       [12.375405 , 69.00971  , 41.239483 , ...,  8.831716 ,  5.223301 ,
         7.893204 ],
       ...,
       [32.826954 , 18.085758 , 11.012252 , ..., 16.411945 , 17.134764 ,
        20.413477 ],
       [39.266953 , 12.705211 ,  8.162741 , ..., 13.226267 ,  4.980728 ,
         5.700214 ],
       [12.268742 ,  9.340878 , 11

In [6]:
# Load vocabulary into BoW
bow = BoW()
bow.load_vocabulary(filename.strip(".pickle"))

# Train the image classifier
image_classifier = ImageClassifier(bow)
print(f"Image classifier: {image_classifier}")
image_classifier.train(training_set, iterations)

# Save the trained classifier
classifier = "classifier"
image_classifier.save(classifier)

print("Vocabulary and classifier successfully built and saved.")


print("Empezando el train: ")

bow = BoW()
# TODO: Especify the args for the loading method
bow.load_vocabulary(filename.strip(".pickle"))

image_classifier = ImageClassifier(bow)
# TODO: Especify the args for the loading method
image_classifier.load(classifier)
# TODO: Especify the args for the loading method
image_classifier.predict(training_set)





Image classifier: <image_classifier.ImageClassifier object at 0x000001E66A397190>


TRAINING CLASSIFIER

Extracting features...
 47%|████▋     | 1166/2474 [00:15<00:16, 79.17image/s]WARN: Issue Loading one label from ../data/traffic_Data/DATA\No stopping\054_0084.png
WARN: Issue Loading one label from ../data/traffic_Data/DATA\No stopping\054_0085.png
 54%|█████▎    | 1327/2474 [00:17<00:14, 78.62image/s] WARN: Issue Loading one label from ../data/traffic_Data/DATA\No stopping\054_1_0084.png
WARN: Issue Loading one label from ../data/traffic_Data/DATA\No stopping\054_1_0085.png
100%|██████████| 2474/2474 [00:36<00:00, 68.59image/s]

Training SVM...
Vocabulary and classifier successfully built and saved.
Empezando el train: 


CLASSIFICATION RESULTS

Confusion matrix

KNOWN/PREDICTED          Bicycles crossing    36  Dont Go Left  Dont overtake from Left  Go Right  Go right or straight  Go straight  ...  No horn  No stopping  Speed limit (5kmh)  Speed limit (60kmh)  Zebra Crossing  spee

(0.8934959349593496,
 array([[142.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.,   6.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  34.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.,   4.,   0.,   0.],
        [  0.,   0., 134.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   2., 120.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   2.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  96.,   0.,   0.,   0.,   2.,   0.,   2.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   2.,   0.,  72.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   6.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.,  10.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   2.,   0.,   0.,   0., 140.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   2.,   0.,   0.,   0., 134.,   0.,  26.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.

In [7]:
print("Empezando el test: ")
bow = BoW()
# TODO: Especify the args for the loading method
bow.load_vocabulary(filename.strip(".pickle"))

image_classifier = ImageClassifier(bow)
# TODO: Especify the args for the loading method
image_classifier.load(classifier)
# TODO: Especify the args for the loading method
image_classifier.predict(validation_set)

Empezando el test: 


CLASSIFICATION RESULTS

Confusion matrix

KNOWN/PREDICTED          Bicycles crossing   36  Dont Go Left  Dont overtake from Left  Go Right  Go right or straight  Go straight  ...  No horn  No stopping  Speed limit (5kmh)  Speed limit (60kmh)  Zebra Crossing  speed limit (80kmh)  watch out for cars
Bicycles crossing                     16.0  0.0           0.0                      0.0       2.0                   8.0          0.0  ...      0.0          8.0                 0.0                  2.0             4.0                  0.0                 4.0
36                                     0.0  0.0           0.0                      0.0       0.0                   0.0          0.0  ...      0.0          0.0                 0.0                  0.0             0.0                  0.0                 0.0
Dont Go Left                           0.0  0.0         100.0                      0.0       6.0                   0.0          0.0  ...      2.0          6.0       

(0.5746887966804979,
 array([[ 16.,   0.,   0.,   0.,   2.,   8.,   0.,   0.,   2.,   0.,   8.,   0.,   2.,   4.,   0.,   4.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0., 100.,   0.,   6.,   0.,   0.,   4.,   2.,   2.,   6.,   4.,   2.,   0.,   0.,   4.],
        [  0.,   0.,   0.,   4.,   0.,   0.,   0.,   0.,   0.,   2.,   2.,   2.,   2.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  16.,   0.,   0.,   4.,   4.,   0.,   0.,   0.,   0.,   2.,   0.,   0.],
        [  6.,   0.,   0.,   0.,  26.,  52.,   0.,  12.,   2.,   0.,   2.,   0.,   4.,  12.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   2.,   0.,   4.,   4.,   0.,   0.,   0.,   0.],
        [  2.,   0.,   2.,   0.,   2.,   0.,   0.,  56.,   8.,   4.,   0.,   2.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   2.,   0.,   4.,   0.,   0.,   0.,  24.,   0.,  26.,   0.,   0.,   0.,   0.,   2.],
        [  2.,   0.,   2.